In [8]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import pandas as pd

In [9]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
sns.set()

# CUSTOM LAYERS

In [10]:
class InputLayer(tf.keras.layers.Layer):
  def __init__(self, name):
    super(InputLayer, self).__init__(name=name)
    self.a = tf.Variable(initial_value=tf.zeros(shape=[14,2]), trainable=False, name= self.name + "_activation")
    
  def call(self, inputs):
    a1a1 = tf.reshape(inputs[0],shape=(1,1))   #1
    a1a2 = tf.reshape(inputs[1],shape=(1,1))   #2 
    a1a2a2 = tf.reshape(inputs[2],shape=(1,1)) #3
    a1a2a1 = tf.reshape(inputs[3],shape=(1,1)) #4
    a2a2 = tf.reshape(inputs[4],shape=(1,1))   #5
    a2a1 = tf.reshape(inputs[5],shape=(1,1))   #6
    a2a1a1 = tf.reshape(inputs[6],shape=(1,1)) #7
    a2a1a2 = tf.reshape(inputs[7],shape=(1,1)) #8

    to_hidden = []
    to_hidden.append((tf.concat([a1a2a2, a2a2], axis = 1)))   #["TrRi"] [3,5]
    to_hidden.append((tf.concat([a1a2a1, a2a1], axis = 1)))   #["RiTr"] [4,6] 
    to_hidden.append((tf.concat([a1a1, a2a1], axis = 1)))     #["Plu"]  [1,6] 
    to_hidden.append((tf.concat([a1a2, a1a2a2], axis = 1)))   #["TTra"] [2,3] 
    to_hidden.append((tf.concat([a2a1, a2a1a1], axis = 1)))   #["TTrb"] [6,7] 
    to_hidden.append((tf.concat([a1a2, a2a1], axis = 1)))     #["Tr"]   [2,6]
    to_hidden.append((tf.concat([a1a2a1, a2a1a2], axis = 1))) #["MeTr"] [4,8] 
    to_hidden.append((tf.concat([a1a2a1, a1a2], axis = 1)))   #["MoTr"] [4,2] 
    to_hidden.append((tf.concat([a1a1, a1a2a1], axis = 1)))   #["RiRi"] [1,4] 
    to_hidden.append((tf.concat([a1a1, a1a2], axis = 1)))     #["MoFo"] [1,2] 
    to_hidden.append((tf.concat([a1a2a2, a1a2a1], axis = 1))) #["MMof"] [3,4] 
    to_hidden.append((tf.concat([a1a1, a1a2a2], axis = 1)))   #["MoRi"] [1,3] 
    to_hidden.append((tf.concat([a1a2a2, a2a1a1], axis = 1))) #["MeRi"] [3,7] 
    to_hidden.append((tf.concat([a1a1, a2a2], axis = 1)))     #["Ri"]   [1,5] 

    output = tf.concat(to_hidden,axis = 0)
    output = tf.transpose(output)
    self.a.assign(output)
    return self.a.value()

In [11]:
class Layer(tf.keras.layers.Layer):
  def __init__(self, name, weights_init_val, bias_init_val):
    super(Layer, self).__init__(name = name)
    self.units = 14
    self.weights_init_val = weights_init_val
    self.bias_init_val = bias_init_val
    if self.name in ["HiddenLayer"]:
      self.type = "Hidden"
    elif self.name in ["OutputLayer"]:
      self.type = "Output"

  def build(self, input_shape):
    shape = [int(input_shape[-1]),self.units]
    self.w = self.add_weight(initializer = tf.initializers.Constant(self.weights_init_val) ,
                              shape= shape,
                              name = self.name + "_weights")
    self.b = self.add_weight(initializer = tf.initializers.Constant(self.bias_init_val) ,
                              shape= [1, self.units],
                              name = self.name + "_biases")

    self.deltaA = tf.Variable(initial_value=tf.zeros(shape = [1, self.units]),trainable=False, name= self.name + "_adjustements")
    self.a = tf.Variable(initial_value=tf.zeros(shape = [1, self.units]),trainable=False, name= self.name + "_activation")

  def call(self, inputs, temperature):
     # i pesi dei neuroni sono considerati in colonna
    if self.type == "Hidden":
      net = tf.math.reduce_sum(inputs * self.w, axis=0) 
     
    elif self.type == "Output":
      net = tf.matmul(inputs,self.w) 
      
    self.a.assign( 1/( 1 + tf.exp( -( (net + self.b)/temperature)) ) )
    return self.a.value()


### Testing foward pass

In [55]:
x = tf.ones(shape=[3,4])
x

<tf.Tensor: shape=(3, 4), dtype=float32, numpy=
array([[1., 1., 1., 1.],
       [1., 1., 1., 1.],
       [1., 1., 1., 1.]], dtype=float32)>

In [13]:
tf.reduce_sum(x, axis=0)

<tf.Tensor: shape=(4,), dtype=float32, numpy=array([3., 3., 3., 3.], dtype=float32)>

In [61]:
y = tf.constant([[0.1, 0.2, 0.3, 0.4]])
y

<tf.Tensor: shape=(1, 4), dtype=float32, numpy=array([[0.1, 0.2, 0.3, 0.4]], dtype=float32)>

In [64]:
x * y 

<tf.Tensor: shape=(3, 4), dtype=float32, numpy=
array([[0.1, 0.2, 0.3, 0.4],
       [0.1, 0.2, 0.3, 0.4],
       [0.1, 0.2, 0.3, 0.4]], dtype=float32)>

In [14]:
x = tf.constant([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8])
#x = tf.constant([0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0])
x

<tf.Tensor: shape=(8,), dtype=float32, numpy=array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8], dtype=float32)>

In [15]:
input_layer = InputLayer(name = "InputLayer")
input_layer

In [16]:
res_input = input_layer(x)
res_input

<tf.Tensor: shape=(14, 2), dtype=float32, numpy=
array([[0.3, 0.5],
       [0.4, 0.6],
       [0.1, 0.6],
       [0.2, 0.3],
       [0.6, 0.7],
       [0.2, 0.6],
       [0.4, 0.8],
       [0.4, 0.2],
       [0.1, 0.4],
       [0.1, 0.2],
       [0.3, 0.4],
       [0.1, 0.3],
       [0.3, 0.7],
       [0.1, 0.5]], dtype=float32)>

In [17]:
input_layer.a

<tf.Variable 'InputLayer_activation:0' shape=(14, 2) dtype=float32, numpy=
array([[0.3, 0.5],
       [0.4, 0.6],
       [0.1, 0.6],
       [0.2, 0.3],
       [0.6, 0.7],
       [0.2, 0.6],
       [0.4, 0.8],
       [0.4, 0.2],
       [0.1, 0.4],
       [0.1, 0.2],
       [0.3, 0.4],
       [0.1, 0.3],
       [0.3, 0.7],
       [0.1, 0.5]], dtype=float32)>

In [18]:
hidden_layer = Layer(name = "HiddenLayer",weights_init_val=0.5, bias_init_val=0.0)
hidden_layer

In [19]:
res_hidden = hidden_layer(inputs = res_input, temperature = 1)
res_hidden

<tf.Tensor: shape=(1, 14), dtype=float32, numpy=
array([[0.59868765, 0.62245935, 0.5866176 , 0.5621765 , 0.65701044,
        0.59868765, 0.6456563 , 0.5744425 , 0.5621765 , 0.5374298 ,
        0.5866176 , 0.54983395, 0.62245935, 0.5744425 ]], dtype=float32)>

In [20]:
hidden_layer.w

<tf.Variable 'HiddenLayer/HiddenLayer_weights:0' shape=(2, 14) dtype=float32, numpy=
array([[0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5,
        0.5],
       [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5,
        0.5]], dtype=float32)>

In [21]:
hidden_layer.a

<tf.Variable 'HiddenLayer/HiddenLayer_activation:0' shape=(1, 14) dtype=float32, numpy=
array([[0.59868765, 0.62245935, 0.5866176 , 0.5621765 , 0.65701044,
        0.59868765, 0.6456563 , 0.5744425 , 0.5621765 , 0.5374298 ,
        0.5866176 , 0.54983395, 0.62245935, 0.5744425 ]], dtype=float32)>

In [22]:
output_layer = Layer(name = "OutputLayer",weights_init_val=0.59, bias_init_val=0.0)
output_layer

In [23]:
res_output = output_layer(inputs = res_hidden, temperature = 1)
res_output

<tf.Tensor: shape=(1, 14), dtype=float32, numpy=
array([[0.99249333, 0.99249333, 0.99249333, 0.99249333, 0.99249333,
        0.99249333, 0.99249333, 0.99249333, 0.99249333, 0.99249333,
        0.99249333, 0.99249333, 0.99249333, 0.99249333]], dtype=float32)>

In [24]:
output_layer.w

<tf.Variable 'OutputLayer/OutputLayer_weights:0' shape=(14, 14) dtype=float32, numpy=
array([[0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59,
        0.59, 0.59, 0.59],
       [0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59,
        0.59, 0.59, 0.59],
       [0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59,
        0.59, 0.59, 0.59],
       [0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59,
        0.59, 0.59, 0.59],
       [0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59,
        0.59, 0.59, 0.59],
       [0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59,
        0.59, 0.59, 0.59],
       [0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59,
        0.59, 0.59, 0.59],
       [0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59,
        0.59, 0.59, 0.59],
       [0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59, 0.59,
        0.59, 0.59, 0.59],
     

In [25]:
output_layer.a

<tf.Variable 'OutputLayer/OutputLayer_activation:0' shape=(1, 14) dtype=float32, numpy=
array([[0.99249333, 0.99249333, 0.99249333, 0.99249333, 0.99249333,
        0.99249333, 0.99249333, 0.99249333, 0.99249333, 0.99249333,
        0.99249333, 0.99249333, 0.99249333, 0.99249333]], dtype=float32)>

In [26]:
error = tf.math.reduce_sum(tf.math.square(res_hidden - res_output))
error

<tf.Tensor: shape=(), dtype=float32, numpy=2.2692394>

### Test back propagation

In [27]:
alpha = 1

In [28]:
delta_Ao = res_output * (res_hidden - res_output) * (1 - res_output) + output_layer.deltaA * (res_hidden - res_output)
delta_Ao

<tf.Tensor: shape=(1, 14), dtype=float32, numpy=
array([[-0.00293398, -0.00275687, -0.0030239 , -0.003206  , -0.00249945,
        -0.00293398, -0.00258405, -0.00311461, -0.003206  , -0.00339037,
        -0.0030239 , -0.00329795, -0.00275687, -0.00311461]],
      dtype=float32)>

In [29]:
res_hidden

<tf.Tensor: shape=(1, 14), dtype=float32, numpy=
array([[0.59868765, 0.62245935, 0.5866176 , 0.5621765 , 0.65701044,
        0.59868765, 0.6456563 , 0.5744425 , 0.5621765 , 0.5374298 ,
        0.5866176 , 0.54983395, 0.62245935, 0.5744425 ]], dtype=float32)>

In [30]:
tf.transpose(res_hidden)

<tf.Tensor: shape=(14, 1), dtype=float32, numpy=
array([[0.59868765],
       [0.62245935],
       [0.5866176 ],
       [0.5621765 ],
       [0.65701044],
       [0.59868765],
       [0.6456563 ],
       [0.5744425 ],
       [0.5621765 ],
       [0.5374298 ],
       [0.5866176 ],
       [0.54983395],
       [0.62245935],
       [0.5744425 ]], dtype=float32)>

In [31]:
delta_Ao

<tf.Tensor: shape=(1, 14), dtype=float32, numpy=
array([[-0.00293398, -0.00275687, -0.0030239 , -0.003206  , -0.00249945,
        -0.00293398, -0.00258405, -0.00311461, -0.003206  , -0.00339037,
        -0.0030239 , -0.00329795, -0.00275687, -0.00311461]],
      dtype=float32)>

In [32]:
delta_Wo = alpha * tf.matmul(tf.transpose(res_hidden),delta_Ao)   #dubbio scambiare righe e colonne
delta_Wo

<tf.Tensor: shape=(14, 14), dtype=float32, numpy=
array([[-0.00175654, -0.0016505 , -0.00181037, -0.00191939, -0.00149639,
        -0.00175654, -0.00154704, -0.00186468, -0.00191939, -0.00202977,
        -0.00181037, -0.00197444, -0.0016505 , -0.00186468],
       [-0.00182628, -0.00171604, -0.00188226, -0.0019956 , -0.00155581,
        -0.00182628, -0.00160846, -0.00193872, -0.0019956 , -0.00211037,
        -0.00188226, -0.00205284, -0.00171604, -0.00193872],
       [-0.00172112, -0.00161723, -0.00177387, -0.00188069, -0.00146622,
        -0.00172112, -0.00151585, -0.00182709, -0.00188069, -0.00198885,
        -0.00177387, -0.00193464, -0.00161723, -0.00182709],
       [-0.00164941, -0.00154985, -0.00169997, -0.00180234, -0.00140513,
        -0.00164941, -0.00145269, -0.00175096, -0.00180234, -0.00190599,
        -0.00169997, -0.00185403, -0.00154985, -0.00175096],
       [-0.00192765, -0.00181129, -0.00198674, -0.00210637, -0.00164217,
        -0.00192765, -0.00169775, -0.00204633, -0

In [33]:
output_layer.w.assign_add(delta_Wo)
output_layer.w

<tf.Variable 'OutputLayer/OutputLayer_weights:0' shape=(14, 14) dtype=float32, numpy=
array([[0.5882434 , 0.58834946, 0.5881896 , 0.5880806 , 0.5885036 ,
        0.5882434 , 0.58845294, 0.5881353 , 0.5880806 , 0.5879702 ,
        0.5881896 , 0.5880255 , 0.58834946, 0.5881353 ],
       [0.5881737 , 0.58828396, 0.5881177 , 0.58800435, 0.5884442 ,
        0.5881737 , 0.5883915 , 0.5880613 , 0.58800435, 0.5878896 ,
        0.5881177 , 0.58794713, 0.58828396, 0.5880613 ],
       [0.58827883, 0.5883827 , 0.5882261 , 0.58811927, 0.58853376,
        0.58827883, 0.5884841 , 0.5881729 , 0.58811927, 0.58801115,
        0.5882261 , 0.5880653 , 0.5883827 , 0.5881729 ],
       [0.58835053, 0.58845013, 0.5883    , 0.58819765, 0.58859485,
        0.58835053, 0.5885473 , 0.588249  , 0.58819765, 0.588094  ,
        0.5883    , 0.588146  , 0.58845013, 0.588249  ],
       [0.5880723 , 0.5881887 , 0.58801323, 0.5878936 , 0.5883578 ,
        0.5880723 , 0.58830225, 0.5879536 , 0.5878936 , 0.5877725 ,
      

In [34]:
delta_Ah = res_hidden * (1 - res_hidden) * (tf.matmul(delta_Ao , output_layer.w)) # dubbio scambiare righe per colonne
delta_Ah

<tf.Tensor: shape=(1, 14), dtype=float32, numpy=
array([[-0.00591398, -0.00578561, -0.0059685 , -0.00605691, -0.00554931,
        -0.00591398, -0.00563346, -0.00601622, -0.00605691, -0.00611644,
        -0.0059685 , -0.00609037, -0.00578561, -0.00601622]],
      dtype=float32)>

In [35]:
delta_Wh = tf.transpose(alpha * tf.transpose(delta_Ah) * res_input)
delta_Wh

<tf.Tensor: shape=(2, 14), dtype=float32, numpy=
array([[-0.0017742 , -0.00231424, -0.00059685, -0.00121138, -0.00332959,
        -0.0011828 , -0.00225339, -0.00240649, -0.00060569, -0.00061164,
        -0.00179055, -0.00060904, -0.00173568, -0.00060162],
       [-0.00295699, -0.00347137, -0.0035811 , -0.00181707, -0.00388452,
        -0.00354839, -0.00450677, -0.00120324, -0.00242276, -0.00122329,
        -0.0023874 , -0.00182711, -0.00404993, -0.00300811]],
      dtype=float32)>

In [36]:
hidden_layer.w.assign_add(delta_Wh)
hidden_layer.w

<tf.Variable 'HiddenLayer/HiddenLayer_weights:0' shape=(2, 14) dtype=float32, numpy=
array([[0.4982258 , 0.49768576, 0.49940315, 0.49878863, 0.49667042,
        0.4988172 , 0.49774662, 0.49759352, 0.4993943 , 0.49938837,
        0.49820945, 0.49939096, 0.4982643 , 0.49939838],
       [0.497043  , 0.49652863, 0.4964189 , 0.49818292, 0.49611548,
        0.49645162, 0.49549323, 0.49879676, 0.49757725, 0.4987767 ,
        0.4976126 , 0.49817288, 0.49595007, 0.4969919 ]], dtype=float32)>

---

# CUSTOM MODEL

In [37]:
class MQSelfReflexiveNetwork(tf.keras.Model):
  def __init__(self, learning_rate):
    super(MQSelfReflexiveNetwork, self).__init__(name='MQSelfReflexiveNetwork')

    self.input_layer = InputLayer(name = "InputLayer")
    self.h = Layer(name = "HiddenLayer",weights_init_val=0.5, bias_init_val=0.1)
    self.o = Layer(name = "OutputLayer",weights_init_val=0.59, bias_init_val=0.1)
    self.temperature = tf.Variable(initial_value=1.0,trainable=False, name = "Temperature")
    self.error = tf.Variable(initial_value=0.0,trainable=False, name = "Error")
    self.alpha = learning_rate

  def call(self, input):
    ### FOWARD PASS
    res_input = self.input_layer(input)
    res_hidden = self.h(res_input, self.temperature)
    res_output = self.o(res_hidden, self.temperature)
    return tf.math.reduce_sum(tf.math.square(res_hidden - res_output))
  
  def train_step(self, input):
    ### CALL FOWARD PASS
    self.error.assign(self(input, training = True))

    ### BACK PROPAGATION
    # Compute temperature
    self.temperature.assign(1 - (1 / (1 + self.error) ))

    # Update outputs weights
    delta_Ao = self.o.a * (self.h.a - self.o.a) * (1 - self.o.a) + (self.o.deltaA * ((self.h.a - self.o.a)) )
    delta_Wo = self.alpha * tf.matmul(tf.transpose(self.h.a),delta_Ao)
    self.o.deltaA.assign(delta_Ao)
    self.o.w.assign_add(delta_Wo)

    ## Update hidden weights
    delta_Ah = self.h.a * (1 - self.h.a) * tf.matmul(self.o.deltaA, self.o.w)
    delta_Wh = self.alpha * delta_Ah * self.input_layer.a
    self.h.deltaA.assign(delta_Ah)
    self.h.w.assign_add(delta_Wh)

    return {
        "Error": tf.squeeze(self.error.value()),
        "Temperature": tf.squeeze(self.temperature.value()),
        "Hidden weights": tf.squeeze(self.h.w.value()),
        "Output weights": tf.squeeze(self.o.w.value())
        }

In [38]:
# Construct an instance of CustomModel
input = tf.constant([0.01, 0.00, 0.03, 0.224, 0.875, 0.266, 0.427, 0.168])
#input = tf.constant([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8])
model = MQSelfReflexiveNetwork(learning_rate=10)

In [39]:
model.compile()
model_history = model.fit(x = input, epochs=500)

Epoch 1/500
Input layer activations
<tf.Variable 'InputLayer_activation:0' shape=(14, 2) dtype=float32>
Input layer activations
<tf.Variable 'InputLayer_activation:0' shape=(14, 2) dtype=float32>
1/1 [==============================] - 1s 668ms/step - Error: 2.5131 - Temperature: 0.7153 - Hidden weights: 0.4888 - Output weights: 0.5706
Epoch 2/500
1/1 [==============================] - 0s 4ms/step - Error: 1.9046 - Temperature: 0.6557 - Hidden weights: 0.4937 - Output weights: 0.5768
Epoch 3/500
1/1 [==============================] - 0s 9ms/step - Error: 1.7691 - Temperature: 0.6389 - Hidden weights: 0.4916 - Output weights: 0.5739
Epoch 4/500
1/1 [==============================] - 0s 3ms/step - Error: 1.7406 - Temperature: 0.6351 - Hidden weights: 0.4920 - Output weights: 0.5744
Epoch 5/500
1/1 [==============================] - 0s 5ms/step - Error: 2.0460 - Temperature: 0.6717 - Hidden weights: 0.4915 - Output weights: 0.5734
Epoch 6/500
1/1 [==============================] - 0s 4ms/s

In [40]:
df_model_history = pd.DataFrame(model_history.history)
df_model_history

,Error,Temperature,Hidden weights,Output weights
0,2.513080,0.715349,"[[0.4979495, 0.48975956, 0.49935108, 0.4844620...","[[0.57048744, 0.57213867, 0.571271, 0.57025266..."
1,1.904614,0.655720,"[[0.5030992, 0.49751228, 0.5049583, 0.4963272,...","[[0.57719076, 0.57759064, 0.577465, 0.5749193,..."
2,1.769146,0.638878,"[[0.50297636, 0.49280465, 0.5013177, 0.4940014...","[[0.57363224, 0.57566154, 0.5747807, 0.5723604..."
3,1.740628,0.635120,"[[0.5035286, 0.49298266, 0.5023315, 0.49460283...","[[0.57458675, 0.57589525, 0.57517636, 0.572799..."
4,2.045986,0.671699,"[[0.5033968, 0.49298266, 0.50189716, 0.4940574...","[[0.5733787, 0.57527447, 0.57460505, 0.5718952..."
...,...,...,...,...
495,3.528860,0.779194,"[[-0.49367043, -2.6153207, 0.5377939, -0.52970...","[[8.191203, -11.6885805, 5.741327, -8.286032, ..."
496,4.297955,0.811248,"[[-0.49405786, -2.6075006, 0.5346935, -0.52967...","[[8.202897, -11.721581, 5.74514, -8.286513, 12..."
497,5.009296,0.833591,"[[-0.49405786, -2.609258, 0.5380265, -0.529789...","[[8.198742, -11.6921, 5.744115, -8.286264, 12...."
498,1.938270,0.659664,"[[-0.537762, -2.609258, 0.4790175, -0.38876036...","[[8.207036, -11.959055, 5.770984, -8.44532, 12..."


In [41]:
model.get_weights()[1][0]

array([-0.537344  , -2.7006738 ,  0.49420673, -0.45911485,  0.9965732 ,
        0.0080332 , -2.4380784 ,  0.4561199 , -0.6009341 , -3.9660103 ,
        2.9522202 , -1.8902292 , -1.6264325 , -0.3120481 ], dtype=float32)

In [42]:
model.get_weights()[1][1]

array([ 1.5499539 , -2.8156843 ,  0.59192574,  0.7298795 , -0.05325098,
       -0.25538096, -3.2674663 , -0.3737477 ,  1.348815  , -3.2373185 ,
       -1.8668265 ,  3.3337317 ,  0.78754646,  0.58492196], dtype=float32)

In [43]:
dvf = abs(model.get_weights()[1][0] - model.get_weights()[1][1])
dvf

array([2.087298  , 0.1150105 , 0.09771901, 1.1889944 , 1.0498242 ,
       0.26341414, 0.8293879 , 0.8298676 , 1.949749  , 0.7286918 ,
       4.819047  , 5.223961  , 2.413979  , 0.89697003], dtype=float32)

In [44]:
model.get_weights()[0]

array([[0.03 , 0.224],
       [0.427, 0.   ],
       [0.168, 0.   ],
       [0.266, 0.03 ],
       [0.   , 0.01 ],
       [0.266, 0.   ],
       [0.427, 0.875],
       [0.427, 0.266],
       [0.168, 0.427],
       [0.168, 0.266],
       [0.03 , 0.427],
       [0.168, 0.03 ],
       [0.03 , 0.01 ],
       [0.168, 0.224]], dtype=float32)

In [45]:
model.get_weights()[0][:,0]

array([0.03 , 0.427, 0.168, 0.266, 0.   , 0.266, 0.427, 0.427, 0.168,
       0.168, 0.03 , 0.168, 0.03 , 0.168], dtype=float32)

In [46]:
model.get_weights()[0][:,1]

array([0.224, 0.   , 0.   , 0.03 , 0.01 , 0.   , 0.875, 0.266, 0.427,
       0.266, 0.427, 0.03 , 0.01 , 0.224], dtype=float32)

In [47]:
dva = abs(model.get_weights()[0][:,0] - model.get_weights()[0][:,1])
dva

array([0.194     , 0.427     , 0.168     , 0.236     , 0.01      ,
       0.266     , 0.448     , 0.16099998, 0.259     , 0.098     ,
       0.39699998, 0.138     , 0.02      , 0.05600001], dtype=float32)

In [48]:
dvf

array([2.087298  , 0.1150105 , 0.09771901, 1.1889944 , 1.0498242 ,
       0.26341414, 0.8293879 , 0.8298676 , 1.949749  , 0.7286918 ,
       4.819047  , 5.223961  , 2.413979  , 0.89697003], dtype=float32)

In [49]:
fig = px.line(df_model_history[["Error","Temperature"]])
fig.update_layout(title="Monitored variables",
                   xaxis_title='Epoch',
                   yaxis_title='',
                   hovermode='x unified')
fig.show()

---